# MOABB Motor Imagery Dataset example

Import the motor imagery (MI) trials from MOABB 

In [1]:
%matplotlib qt

## Import libraries
import mne
import sys
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from mne.decoding import CSP
from mne.decoding import PSDEstimator
from moabb.datasets import BNCI2015004
from moabb.paradigms import MotorImagery 
import matplotlib.collections as collections
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
sys.path.append("..") # Adds higher directory to python modules path.
from Functions import artifact_removal_tools as art
from Functions import eeg_preprocessing
from Functions import eeg_quality_index
from Functions import art_pipeline
from Functions import psd_auc
# from ..Functions import psd_auc
from moabb.evaluations import WithinSessionEvaluation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

## Import MI data



In [2]:
# Settings
subjects = [1]    # List of subjects [n]

# Import dataset
dataset = BNCI2015004()
dataset.subject_list = subjects

# Select session from dataset
sessions = dataset.get_data(subjects=subjects)

In [3]:
# Select MI paradigm for classes (n=5): Word, Subtaction, Navigation, RightHand, and Feet
fc = [0.1, 100] # Cut-off frequencies [Hz]
# fc = [8, 30] # Cut-off frequencies [Hz]
# chans = ['FC3', 'FCz', 'FC4',
#         'C3', 'Cz', 'C4',
#         'CP3', 'CPz', 'CP4']    # List of str channels to use
# chans = ['Fz']
tmax = 6    # Time to end epoch [sec]
# labels = ['right_hand', 'feet', 'navigation', 'subtraction', 'word_ass']    # List of str of desired events
labels = ['right_hand', 'feet']    # List of str of desired events
paradigm = MotorImagery(n_classes=len(labels), events=labels, fmin=fc[0], fmax=fc[1], channels=chans, tmax=tmax)

[x, y, metadata] = paradigm.get_data(dataset=dataset, subjects=subjects)

## Create pipelines

Create scikit-learn pipelines for the raw data, and the data processed with the artifact removal tool

In [4]:
# raw_pipeline = make_pipeline(CSP(n_components=8), LDA())
# ART_pipeline = make_pipeline(art_pipeline.ART(srate=256), CSP(n_components=8), LDA())  # Sample rate obtained from documentation
# ART_pipeline = make_pipeline(CSP(n_components=8), LDA())  # Sample rate obtained from documentation

srate = 256
mu_band = [7.5, 12.5]
raw_pipeline = make_pipeline(PSDEstimator(float(srate), fmin=mu_band[0], fmax=mu_band[1], verbose=False), psd_auc.PSD_AUC(),
                            SVC())
# ART_pipeline = make_pipeline(PSDEstimator(float(srate), fmin=mu_band[0], fmax=mu_band[1], verbose=False),
#                             SVC())

# raw_pipeline = make_pipeline(LDA())
# ART_pipeline = make_pipeline(art_pipeline.ART(srate=256), CSP(n_components=8), LDA())  # Sample rate obtained from documentation
# ART_pipeline = make_pipeline(LDA())  # Sample rate obtained from documentation

In [5]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=30)
skf.get_n_splits(x, y)
for train_index, test_index in skf.split(x,y):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    raw_pipeline.fit(x_train, y_train)
    score = raw_pipeline.score(x_test, y_test) 
    raw_predict = raw_pipeline.predict(x_test)

    print(f'Score = {score:0.4f}')
    cfm = confusion_matrix(y_test, raw_predict)
    print(f'Confusion Matrix\n {cfm}')
    

TRAIN: [  0   1   2   3   5   6   7   8   9  10  11  12  13  14  16  17  18  19
  20  22  23  27  29  30  31  32  33  34  35  36  38  39  40  41  42  44
  45  46  48  49  50  51  52  53  54  55  56  57  59  60  61  62  63  64
  65  66  67  68  69  71  72  74  75  76  77  78  81  82  84  85  86  87
  88  89  90  91  93  94  95 103 104 105 106 107 108 110 111 112 113 114
 116 117 118 119 120 121 123 124 125 126 127 128 129 130 131 132 133 135
 136 138 139 140 141 142 143 145 147 148 149 151 152 153 154 155 156 157
 158 159] TEST: [  4  15  21  24  25  26  28  37  43  47  58  70  73  79  80  83  92  96
  97  98  99 100 101 102 109 115 122 134 137 144 146 150]
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
Score = 0.5312
Confusion Matrix
 [[ 5 11]
 [ 4 12]]
TRAIN: [  1   2   3   4   5   6   8   9  10  12  13  14  15  16  17  18  19  21
  22  23  24  25  26

In [6]:
srate = 256
w_length = 3*srate
[x_art, _, _] = art.remove_eyeblinks_cpu(x, srate=srate, n_clusters=10, window_length=w_length)


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=30)
skf.get_n_splits(x_art, y)
for train_index, test_index in skf.split(x_art,y):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x_art[train_index], x_art[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    raw_pipeline.fit(x_train, y_train)
    score = raw_pipeline.score(x_test, y_test) 
    raw_predict = raw_pipeline.predict(x_test)

    print(f'Score = {score:0.4f}')
    cfm = confusion_matrix(y_test, raw_predict)
    print(f'Confusion Matrix\n {cfm}')

TRAIN: [  0   1   2   3   5   6   7   8   9  10  11  12  13  14  16  17  18  19
  20  22  23  27  29  30  31  32  33  34  35  36  38  39  40  41  42  44
  45  46  48  49  50  51  52  53  54  55  56  57  59  60  61  62  63  64
  65  66  67  68  69  71  72  74  75  76  77  78  81  82  84  85  86  87
  88  89  90  91  93  94  95 103 104 105 106 107 108 110 111 112 113 114
 116 117 118 119 120 121 123 124 125 126 127 128 129 130 131 132 133 135
 136 138 139 140 141 142 143 145 147 148 149 151 152 153 154 155 156 157
 158 159] TEST: [  4  15  21  24  25  26  28  37  43  47  58  70  73  79  80  83  92  96
  97  98  99 100 101 102 109 115 122 134 137 144 146 150]
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
Score = 0.4688
Confusion Matrix
 [[ 2 14]
 [ 3 13]]
TRAIN: [  1   2   3   4   5   6   8   9  10  12  13  14  15  16  17  18  19  21
  22  23  24  25  26

: 

In [ ]:
no_art = np.arra

In [ ]:
[x_train, x_test, y_train, y_test] = train_test_split(x, y, test_size=0.4, random_state=30, stratify=y)

raw_pipeline.fit(x_train, y_train)
score = raw_pipeline.score(x_test, y_test) 
raw_predict = raw_pipeline.predict(x_test)

print(f'Score = {score:0.4f}')
cfm = confusion_matrix(y_test, raw_predict)
print(f'Confusion Matrix\n {cfm}')

In [ ]:
srate = 256
w_length = 3*srate
[x_art_train, _, _] = art.remove_eyeblinks_cpu(x_train, srate=srate, n_clusters=10, window_length=w_length)
[x_art_test, _, _] = art.remove_eyeblinks_cpu(x_test, srate=srate, n_clusters=10, window_length=w_length)

ART_pipeline.fit(x_art_train, y_train)
ART_pipeline.score(x_art_test, y_test)


In [ ]:
art_predict = ART_pipeline.predict(x_art_test)

confusion_matrix(y_test, art_predict)

## Evaluate data

## Evaluate raw data

Evaluate the data without using the artifact removal tool

In [ ]:
evaluation = WithinSessionEvaluation(
    paradigm=paradigm,
    datasets=[dataset],
    overwrite=True,
    hdf5_path=None,
)

raw_results = evaluation.process({"csp+lda": raw_pipeline})

## Evaluate ART data

In [ ]:
# [x_art, _, _] = art.remove_eyeblinks_cpu(x, srate=256, window_length=512)

In [ ]:
art_results = evaluation.process({"art+csp+lda": ART_pipeline}) 

In [ ]:
x_train, x_test, y_train2, y_test = train_test_split(x, y, test_size=0.9, random_state=30, stratify=y)
# From here pick x_train and y_train to do ART

In [ ]:
z = x[0:33,:,:]
a.info['sfreq']
srate = int(a.info['sfreq'])
[x_art,_,_] = art.remove_eyeblinks_cpu(z, srate, window_length=3*srate)

In [ ]:
epoch = 15
chan = 3
fig, ax = plt.subplots(2,1)
ax[0].plot(z[epoch,chan,:])
ax[1].plot(x_art[epoch,chan,:])

In [ ]:
b =a.info['ch_names']

In [ ]:
pipeline = make_pipeline(art.remove_eyeblinks_cpu(srate, window=3*srate), CSP(n_components=30), LDA())

evaluation = WithinSessionEvaluation(
    paradigm=paradigm,
    datasets=[dataset],
    overwrite=True,
    hdf5_path=None,
)

results2 = evaluation.process({"csp+lda": pipeline})